# A Machine Learning journey from customer reviews to business insights
# *Part 1: Data preparation*

*Author: Federica Lionetto*  
*Email: federica.lionetto@gmail.com*  
*Date: 17 November 2020*  
*License: Creative Commons BY-NC-SA*

*Based on the dataset available at:*
- https://www.kaggle.com/efehandanisman/skytrax-airline-reviews

### First dataset in brief

The first dataset is scraped from Skytrax (https://www.airlinequality.com), an international air transport rating organization based in the UK, whose mission is to improve the customer experience for airlines and airports across the world.  
The dataset consists of individual reviews left by verified customers of most of the major airlines across the world and can be downloaded from https://www.kaggle.com/efehandanisman/skytrax-airline-reviews.  

An individual review contains part or all of the following information:
- **author**
- **review date**
- **route**
- **date flown**
- **type of traveller**
- **service class**
- **overall score** of the travel experience
- **subscores** related to specific aspects of the travel experience, in particular:
  - food and beverage
  - seat comfort 
  - cabin service 
  - ground service
  - entertainment
  - value for money
- **whether the customer would recommend the travel experience to other customers**
- **comments** in a free text format

### Further readings

- "What can we learn from five-star airlines: a web scraping project from Skytrax", https://nycdatascience.com/blog/student-works/web-scraping/what-can-we-learn-from-five-star-airlines-a-web-scraping-project-from-skytrax/


## 1 - Import modules and helper functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('Set2')

import datetime as dt
import dateutil

import importlib

In [ ]:
# Debugging capabilities.
import pdb

In [ ]:
import sys  
sys.path.insert(0, './helper_functions')

In [ ]:
# Related to review date.
import get_review_date_timestamp
import get_review_date_day, get_review_date_month, get_review_date_year
import get_review_date_sec_epoch

# Related to date flown. 
import get_date_flown_timestamp
import get_date_flown_day, get_date_flown_month, get_date_flown_year
import get_date_flown_sec_epoch

# Related to review date and date flown.
import get_review_date_date_flown_distance_days

# Related to route.
import has_layover
import get_origin
import get_destination
import get_layover

# Related to review text.
import get_review_characters, get_review_words

# Related to review score.
import get_pos_neu_neg_review_score

# Related to recommendation.
import assign_label_recommended
import get_recommendation_bool

# Related to missing values.
import get_completeness

# Related to visualization.
import plot_hist
import plot_hist_sns
import plot_two_hists_comp_sns
import plot_bar
import plot_grouped_box
import plot_cmap

## 2 - Load the input data

In [ ]:
# Type of each field in the input data.
df_dtype = {'airline':str,
            'overall':'Int64',
            'author':str,
            'review_date':str,
            'customer_review':str,
            'aircraft':str,
            'traveller_type':str,
            'cabin':str,
            'route':str,
            'date_flown':str,
            'seat_comfort':'Int64',
            'cabin_service':'Int64',
            'food_bev':'Int64',
            'entertainment':'Int64',
            'ground_service':'Int64',
            'value_for_money':'Int64',
            'recommended':str,
           }

In [ ]:
df = pd.read_excel('../Data/capstone_airline_reviews3.xlsx', dtype=df_dtype)

In [ ]:
df.head()

In [ ]:
df.shape

Get the names of the colums in the dataset.

In [ ]:
cols = df.columns.to_list()
print('Columns in the dataset:')
print(cols)

Get the total number of customer reviews in the dataset.

In [ ]:
n_reviews = df.shape[0]
print('Number of customer reviews in the dataset: {:d}'.format(n_reviews))

## 3 - Quick fixes of the input data

**DISCUSSION**:  
*Spend the next 5-10 minutes looking at the dataset and writing down all aspects related to data quality that, in your opinion, should be taken into account before using the dataset for training a Machine Learning model.*

### 3.1 - Text of the customer review

The preview of the dataset shows that the customer review field usually starts with a special character and a default text, followed by the same information displayed in the route field. We can strip this information away from the customer review field as it does not add any additional value. 

In [ ]:
df[['customer_review','route']].head()

In [ ]:
df.iloc[1,:]['customer_review']

In [ ]:
df.iloc[1,:]['route']

In [ ]:
df_quick_fixes = df.copy()

In [ ]:
# Split the string by the first occurrence of '.' and get the second part of the string.
df_quick_fixes['customer_review_clean'] = df_quick_fixes['customer_review'].str.split('.',n=1,expand=True)[1]

In [ ]:
df_quick_fixes['customer_review_clean'].head()

In [ ]:
df_quick_fixes.drop(columns=['customer_review'], inplace=True)

In [ ]:
df_quick_fixes.head()

### 3.2 - Rename and drop columns

For simplicity, we rename some of the fields.

In [ ]:
df_quick_fixes.rename(columns={'overall':'review_score', 'customer_review_clean':'review_text'}, inplace=True)

We drop the author field as it is not relevant for our use case.

In [ ]:
df_quick_fixes.drop(columns=['author'], inplace=True)

In [ ]:
df_quick_fixes.head()

## 4 - Feature engineering

We add new features to the dataset in view of model training and testing.

In [ ]:
df_aug = df_quick_fixes.copy()

### 4.1 - Handling of dates

What is the format of the review date and date flown?  
Can we add any feature related to dates that could be used in a Machine Learning model?

In [ ]:
df_aug[['review_date','date_flown']].dtypes

In [ ]:
df_aug[['review_date','date_flown']].head()

The following features are added to the dataset:
- `review_date_timestamp`
- `review_date_day`
- `review_date_month`
- `review_date_year`
- `review_date_sec_epoch`
- `date_flown_timestamp`
- `date_flown_day`
- `date_flown_month`
- `date_flown_year`
- `date_flown_sec_epoch`
- `review_date_date_flown_distance_days`

#### Review date

In [ ]:
df_aug['review_date'].head()

In [ ]:
# Add new features.
df_aug['review_date_timestamp'] = df_aug.apply(lambda x: get_review_date_timestamp.get_review_date_timestamp(x), axis=1)

In [ ]:
df_aug[['review_date','review_date_timestamp']].head()

In [ ]:
df_aug['review_date_timestamp'].min()

In [ ]:
df_aug['review_date_timestamp'].max()

In [ ]:
# Add new features.
df_aug['review_date_day'] = df_aug.apply(lambda x: get_review_date_day.get_review_date_day(x),axis=1)
df_aug['review_date_month'] = df_aug.apply(lambda x: get_review_date_month.get_review_date_month(x),axis=1)
df_aug['review_date_year'] = df_aug.apply(lambda x: get_review_date_year.get_review_date_year(x),axis=1)

In [ ]:
df_aug[['review_date','review_date_timestamp','review_date_day','review_date_month','review_date_year']].head()

In [ ]:
# Add new features.
df_aug['review_date_sec_epoch'] = df_aug.apply(lambda x: get_review_date_sec_epoch.get_review_date_sec_epoch(x),axis=1)

In [ ]:
df_aug[['review_date','review_date_timestamp','review_date_day','review_date_month','review_date_year','review_date_sec_epoch']].head()

#### Date flown

In [ ]:
df_aug['date_flown'].head()

In [ ]:
# Add new features.
df_aug['date_flown_timestamp'] = df_aug.apply(lambda x: get_date_flown_timestamp.get_date_flown_timestamp(x), axis=1)

In [ ]:
df_aug[['date_flown','date_flown_timestamp']].head()

In [ ]:
# Add new features.
df_aug['date_flown_day'] = df_aug.apply(lambda x: get_date_flown_day.get_date_flown_day(x),axis=1)
df_aug['date_flown_month'] = df_aug.apply(lambda x: get_date_flown_month.get_date_flown_month(x),axis=1)
df_aug['date_flown_year'] = df_aug.apply(lambda x: get_date_flown_year.get_date_flown_year(x),axis=1)

In [ ]:
df_aug[['date_flown','date_flown_timestamp','date_flown_day','date_flown_month','date_flown_year']].head()

In [ ]:
# Add new features.
df_aug['date_flown_sec_epoch'] = df_aug.apply(lambda x: get_date_flown_sec_epoch.get_date_flown_sec_epoch(x),axis=1)

In [ ]:
df_aug[['date_flown','date_flown_timestamp','date_flown_day','date_flown_month','date_flown_year','date_flown_sec_epoch']].head()

#### Time distance in days between review date and date flown

In [ ]:
# Add new features.
df_aug['review_date_date_flown_distance_days'] = df_aug.apply(lambda x: get_review_date_date_flown_distance_days.get_review_date_date_flown_distance_days(x),axis=1)

In [ ]:
df_aug['review_date_date_flown_distance_days'].dtype

In [ ]:
df_aug[['date_flown_timestamp','review_date_timestamp','review_date_date_flown_distance_days']].head()

### 4.2 - Customer review completeness

The following feature is added to the dataset:
- `is_complete`  

This feature tells us if the customer review is complete, that is, if all the fields contain some information.

In [ ]:
# Add new features.
df_aug['is_complete'] = df_aug.apply(lambda x: get_completeness.get_completeness(x),axis=1)

In [ ]:
df_aug['is_complete'].dtype

In [ ]:
df_aug['is_complete'].head()

### 4.3 - Customer review length

The following features are added to the dataset:
- `review_characters`, which corresponds to the number of characters in the customer review
- `review_words`, which corresponds to the number of words in the customer review

In [ ]:
df_aug['review_text'][3]

In [ ]:
# Add new features.
df_aug['review_characters'] = df_aug.apply(lambda x: get_review_characters.get_review_characters(x),axis=1)
df_aug['review_words'] = df_aug.apply(lambda x: get_review_words.get_review_words(x),axis=1)

In [ ]:
df_aug['review_characters'].head()

In [ ]:
df_aug['review_words'].head()

### 4.4 - Review score category

The following feature is added to the dataset:
- `pos_neu_neg_review_score`

This feature has three possible values:
- `neg`, which stands for negative, if the review score is between 1 and 4
- `neu`, which stands for neutral, if the review score is between 5 and 6
- `pos`, which stands for positive, if the review score is between 7 and 10

In [ ]:
df_aug['review_score'].unique()

In [ ]:
df_aug['review_score'].describe()

In [ ]:
# Add new features.
df_aug['pos_neu_neg_review_score'] = df_aug.apply(lambda x: get_pos_neu_neg_review_score.get_pos_neu_neg_review_score(x),axis=1)

In [ ]:
df_aug['pos_neu_neg_review_score'].dtype

In [ ]:
df_aug[['review_score','pos_neu_neg_review_score']].head()

### 4.5 - Consolidation of aircraft names

**DISCUSSION**:  
*How would you consolidate the aircraft names?*

In [ ]:
print('Number of unique values: {:d}'.format(df_aug['aircraft'].nunique()))
df_aug.groupby(['aircraft']).size().sort_values(ascending=False).reset_index(name='frequency')

For simplicity, we drop the feature `aircraft`.

In [ ]:
df_aug.drop(columns=['aircraft'],inplace=True)

### 4.6 - Consolidation of route names

**DISCUSSION**:  
*How would you consolidate the route names?  
Could you engineer some new features corresponding to origin and destination airports, as well as layovers?  
How would you do it in a "smart" way (no brute force approaches)?*

In [ ]:
df_aug['route'].head()

In [ ]:
df_aug['origin'] = df_aug.apply(lambda x: get_origin.get_origin(x), axis=1)
df_aug['destination'] = df_aug.apply(lambda x: get_destination.get_destination(x), axis=1)
df_aug['layover'] = df_aug.apply(lambda x: get_layover.get_layover(x), axis=1)

In [ ]:
df_aug[['route','origin','destination','layover']].head()

In [ ]:
df_aug['has_layover'] = df_aug.apply(lambda x: has_layover.has_layover(x), axis=1)

In [ ]:
df_aug[['route','origin','destination','layover','has_layover']].head()

In [ ]:
df_aug.shape

In [ ]:
df_aug[df_aug['has_layover']==False].shape

In [ ]:
df_aug[df_aug['has_layover']==True].shape

In [ ]:
df_aug['origin'].unique().tolist()

We can categorise the information specified in the route field into four different fields, corresponding to origin, destination, layover and whether the flight has a layover.  
However, we see that origin, destination and layover are still written according to multiple formats, for example with the name of the city, with the name of the airport, or with the airport code. For the Machine Learning model, each way of writing the origin and destination represents a different entity, i.e. a different value of these features. In order to properly use these features within a Machine Learning model, we should first consolidate the possible values and make sure that one convention is followed throughout the dataset. This is out of scope for the current exercise, but it is important to keep it in mind.

We drop the features `route`, `origin`, `destination` and `layover`.  

In [ ]:
df_aug.drop(columns=['route','origin','destination','layover'],inplace=True)

### 4.7 - Recommendation

In [ ]:
df_aug['recommended'].unique()

In [ ]:
df_aug['recommended_bool'] = df_aug.apply(lambda x: get_recommendation_bool.get_recommendation_bool(x), axis=1)

In [ ]:
df_aug[['recommended','recommended_bool']].head()

In [ ]:
df_aug.drop(columns=['recommended'], inplace=True)
df_aug.rename(columns={'recommended_bool':'recommended'}, inplace=True)

In [ ]:
df_aug.head()

### 4.8 - A quick look at all the features

In [ ]:
cols_aug = df_aug.columns.to_list()
print('Columns in the augmented dataset:')
print(cols_aug)

In [ ]:
df_aug.head()

In [ ]:
df_aug.drop(columns=['review_date','date_flown','review_date_timestamp','review_date_sec_epoch','date_flown_timestamp','date_flown_sec_epoch'], inplace=True)

In [ ]:
df_aug.head()

## 5 - Filling rates

Depending on which Machine Learning algorithm we plan to use, we might want to drop records that contain missing values.  
As a first step, we look at the filling rates of each feature in the dataset, to get a feeling of what is available and how often.

### 5.1 - First iteration

In [ ]:
# Percentage of non-null values.
filling_rates = 100.*df_aug.count().sort_values(ascending=False)/df_aug.shape[0]
print(filling_rates)

In [ ]:
values_filling_rates = filling_rates.values
# print(values_filling_rates)
text_filling_rates = filling_rates.index.to_list()
# print(text_filling_rates)

In [ ]:
plt.figure(figsize=(6,6),dpi=100)
sns.set(style="whitegrid")
ax = sns.barplot(x=values_filling_rates, y=text_filling_rates,color="Red")
ax.set(xlabel='Filling percentage (%)', ylabel='Feature')
plt.tight_layout()
plt.savefig('../Results/01/FillingRates1.png')
plt.show()

We notice that there is a subset of customer reviews for which both the airline and the review date are missing. These records will be dropped from our dataset as they likely indicate a data quality issue.

In [ ]:
df_aug[df_aug['airline'].notna()].shape

In [ ]:
df_aug[df_aug['airline'].isna()].shape

In [ ]:
df_without_na_1 = df_aug[df_aug['airline'].notna()].copy()

In [ ]:
df_without_na_1.shape

In [ ]:
df_without_na_1.head()

### 5.2 - Second iteration

In [ ]:
# Percentage of non-null values.
filling_rates_without_na_1 = 100.*df_without_na_1.count().sort_values(ascending=False)/df_without_na_1.shape[0]
print(filling_rates_without_na_1)

In [ ]:
values_filling_rates_without_na_1 = filling_rates_without_na_1.values
# print(values_filling_rates)
text_filling_rates_without_na_1 = filling_rates_without_na_1.index.to_list()
# print(text_filling_rates)

In [ ]:
plt.figure(figsize=(6,6),dpi=100)
sns.set(style="whitegrid")
ax = sns.barplot(x=values_filling_rates_without_na_1, y=text_filling_rates_without_na_1,color="Red")
ax.set(xlabel='Filling percentage (%)', ylabel='Feature')
plt.tight_layout()
plt.savefig('../Results/01/FillingRates2.png')
plt.show()

At this point, we might proceed in different ways, based on the required tradeoff between dataset size and dataset quality/completeness. The best way to go depends on the specific problem at hand. For example, if we plan to train a complex Machine Learning model (e.g., a Deep Neural Network) that can handle missing values, we might prefer to keep the dataset as it is to use as many records as possible in the training phase. On the other hand, if we plan to train a simple Machine Learning model (e.g., a Decision Tree), we might prefer to drop the missing values as we do not expect the Machine Learning model to need million data points.  
Here, we decide to opt for the highest data quality and completeness, so we keep only records for which all the information is available. 

In [ ]:
df_without_na_2 = df_without_na_1.dropna(axis=0,how='any').copy()

In [ ]:
df_without_na_2.shape

In [ ]:
df_without_na_2.head()

In [ ]:
n_reviews_without_na_2 = df_without_na_2.shape[0]
print('Number of customer reviews in the dataset, excluding all missing values: {:d}'.format(n_reviews_without_na_2))

In [ ]:
f_reviews_without_na_2 = n_reviews_without_na_2/n_reviews
print('Fraction of the original dataset: {:f}'.format(f_reviews_without_na_2))

## 6 - Overview of the different types of features

In [ ]:
df_without_na_2.dtypes

### 6.1 Boolean features

In [ ]:
df_for_training = df_without_na_2.copy()

In [ ]:
df_for_training['is_complete'].unique()

In [ ]:
df_for_training.drop(columns=['is_complete'], inplace=True)

In [ ]:
cols_for_training = df_for_training.columns.to_list()

In [ ]:
feats_bool = ['recommended',
              'has_layover']

### 6.2 Categorical and numerical features

In [ ]:
feats_cat = ['airline',
             'traveller_type',
             'cabin','review_text',
             'pos_neu_neg_review_score']

In [ ]:
feats_num = [feat for feat in cols_for_training if feat not in feats_bool and feat not in feats_cat]

### 6.3 - Summary

In [ ]:
print('Boolean features: \n{}\n'.format(feats_bool))
print('Categorical features: \n{}\n'.format(feats_cat))
print('Numerical features: \n{}\n'.format(feats_num))

## 7 - Correlation matrix for numerical features

What is the correlation between numerical features? Can we observe any interesting pattern here?

In [ ]:
corr_values = df_for_training[feats_num].dropna(axis=0,how='any').corr()

In [ ]:
corr_values

In [ ]:
plot_cmap.plot_cmap(matrix_values=corr_values, 
          figsize_w=12, 
          figsize_h=12, 
          filename='../Results/01/Corr.png')

We observe three main things here:
- a positive correlation between the different types of review scores and subscores
- a negative correlation between the length of the review text and the value of the different types of review scores and subscores
- the similarity between using the number of characters and the number of words, from which we conclude that we can drop one of the two features

The correlation of the review subscores with the overall review score gives important indications on which aspects are particularly valued by customers. For example, the review subscore that has the highest correlation with the overall review score is the value for money, while the review subscore that has the lowest correlation with the overall review score is the entertainment. This could suggest that value for money is what customers value, while entertainment is not so critical for a good travel experience. The other review subscores (food and beverage, seat comfort, cabin service and ground service) all have a similar correlation with the overall review score.

In [ ]:
df_for_training_small = df_for_training.drop(columns=['review_words']).copy()

In [ ]:
cols_for_training_small = df_for_training_small.columns.to_list()

In [ ]:
feats_num_for_training_small = [feat for feat in cols_for_training_small if feat not in feats_bool and feat not in feats_cat]

In [ ]:
print('Boolean features: \n{}\n'.format(feats_bool))
print('Categorical features: \n{}\n'.format(feats_cat))
print('Numerical features: \n{}\n'.format(feats_num_for_training_small))

## 8 - Distinct values for categorical features

How many unique values exist for each feature?

In [ ]:
for col in df_for_training_small:
    print('Column:', col)
    print('Number of unique values:', df_for_training_small[col].nunique())
    if (col != 'review_text' and col != 'review_date_date_flown_distance_days' and col != 'review_characters'):
        print(df_for_training_small[col].unique())
    print('')

In [ ]:
for col in df_for_training_small:
    print('Column:', col)
    print('Number of unique values:', df_for_training_small[col].nunique())
    if (col != 'review_text' and col != 'review_date_date_flown_distance_days' and col != 'review_characters'):
        print(df_for_training_small.groupby([col]).size().sort_values(ascending=False))
    print('')

In [ ]:
# Bar plot of occurrences of each value of each categorical feature.
for feat in feats_cat:
    if (feat != 'review_text' and feat != 'origin_city' and feat != 'destination_city' and feat != 'layover_city'):
        if (feat == 'airline'):
            plot_bar.plot_bar(df=df_for_training_small,
                              feat=feat,
                              filename='../Results/01/CatFeatureValues-'+feat+'.png',
                              horizontal=True,
                              figsize_w=15,
                              figsize_h=15)
        else:
            plot_bar.plot_bar(df=df_for_training_small,
                              feat=feat,
                              filename='../Results/01/CatFeatureValues-'+feat+'.png',
                              horizontal=True,
                              figsize_w=None,
                              figsize_h=None)

Out of the total number of selected customer reviews (24563):
- There are 81 distinct airlines, assuming the same airline does not have different abbreviations or ways to spell their name. The airlines are not equally represented in the dataset. The most represented airline is Cathay Pacific Airways and the least represented airline is QantasLink.
- The review dates cover 31 days, 12 months and 5 years, from 2015 to 2019.
- The dates flown cover 2 days, 12 months and 7 years, from 2013 to 2019. This means that passengers can submit a review several years after the date flown. Would it make sense to restrict the dataset to customer reviews submitted soon after the date flown? What could be the implications?
- There are 22814 distinct review texts but 24563 selected customer reviews. This means that some reviews might be identical. This should be investigated more in detail.
- The review score ranges from 1 to 10.
- The review subscores range from 1 to 5.
- There are 4 distinct traveller types (from the most represented to the least, solo leisure, couple leisure, business and family leisure).
- There are 4 distinct cabins (from the most represented to the least, economy class, business class, premium economy and first class).

## 9 - Reviews having the same review text

We start by counting how many times each review text appears in the dataset. We sort the corresponding dataframe by number of customer reviews, in descending order.

In [ ]:
df_same_review_text = df_for_training_small.groupby('review_text')['review_text'].count().reset_index(name='n_reviews').sort_values(by='n_reviews',ascending=False)

We select only the review texts that appear more than once in the dataset.

In [ ]:
df_same_review_text[df_same_review_text['n_reviews']>1]

The first case corresponds to customer reviews for which the customers did not leave any review text. Apart from the missing review text, these seems to be valid customer reviews and there is no apparent reason for discarding them.

In [ ]:
df_for_training_small[df_for_training_small['review_text']=='']

What about the second customer review with duplicated values in the dataset?

In [ ]:
df_for_training_small[df_for_training_small['review_text']==df_same_review_text.loc[5845,:]['review_text']]

In [ ]:
df_for_training_small[df_for_training_small['review_text']==df_same_review_text.loc[5845,:]['review_text']][feats_bool]

In [ ]:
df_for_training_small[df_for_training_small['review_text']==df_same_review_text.loc[5845,:]['review_text']][feats_num_for_training_small]

In [ ]:
df_for_training_small[df_for_training_small['review_text']==df_same_review_text.loc[5845,:]['review_text']][feats_cat]

Here it seems that a passenger left multiple customer reviews, identical to each other. These duplicates must be dropped from the dataset before model training, as they do not add any useful information.

In [ ]:
df_no_duplicates = df_for_training_small.drop_duplicates().copy()

In [ ]:
n_reviews_no_duplicates = df_no_duplicates.shape[0]
print('Number of customer reviews in the dataset, excluding all missing values and dropping duplicates: {:d}'.format(n_reviews_no_duplicates))

In [ ]:
f_reviews_no_duplicates = n_reviews_no_duplicates/n_reviews
print('Fraction of the original dataset: {:f}'.format(f_reviews_no_duplicates))

## 10 - Distribution of features and uniformity over time

### 10.1 - Review score and selection bias

What is the distribution of the review score? Are customers with extreme feedback more likely to give their feedback?

In [ ]:
feat = 'review_score'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat_x = 'recommended'
feat_y = 'review_score'
plot_grouped_box.plot_grouped_box(df_no_duplicates,feat_x,feat_y,'../Results/01/Box-'+feat_x+'-'+feat_y+'.png')

In [ ]:
feat_x = 'review_score'
feat_y = 'traveller_type'
plot_grouped_box.plot_grouped_box(df_no_duplicates,feat_x,feat_y,'../Results/01/Box-'+feat_x+'-'+feat_y+'.png')

In [ ]:
feat_x = 'review_score'
feat_y = 'cabin'
plot_grouped_box.plot_grouped_box(df_no_duplicates,feat_x,feat_y,'../Results/01/Box-'+feat_x+'-'+feat_y+'.png')

### 10.2 - Review subscores

In [ ]:
feat = 'seat_comfort'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat_x = 'seat_comfort'
feat_y = 'cabin'
plot_grouped_box.plot_grouped_box(df_no_duplicates,feat_x,feat_y,'../Results/01/Box-'+feat_x+'-'+feat_y+'.png')

In [ ]:
feat = 'cabin_service'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat_x = 'cabin_service'
feat_y = 'cabin'
plot_grouped_box.plot_grouped_box(df_no_duplicates,feat_x,feat_y,'../Results/01/Box-'+feat_x+'-'+feat_y+'.png')

In [ ]:
feat = 'food_bev'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat_x = 'food_bev'
feat_y = 'cabin'
plot_grouped_box.plot_grouped_box(df_no_duplicates,feat_x,feat_y,'../Results/01/Box-'+feat_x+'-'+feat_y+'.png')

In [ ]:
feat = 'entertainment'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat = 'ground_service'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat = 'value_for_money'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

### 10.3 - Review date

In [ ]:
feat = 'review_date_day'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat = 'review_date_month'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat = 'review_date_year'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

### 10.4 - Date flown

In [ ]:
feat = 'date_flown_day'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat = 'date_flown_month'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

In [ ]:
feat = 'date_flown_year'
plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

### 10.5 - Distance in days from date flown to review date

In [ ]:
feat = 'review_date_date_flown_distance_days'
plot_hist.plot_hist(df=df_no_duplicates,
          feat=feat,
          bins=100,
          x_min=-100,
          x_max=400,
          filename='../Results/01/Hist-'+feat+'.png')

### 10.6 - Length of the review text

In [ ]:
feat = 'review_characters'
plot_hist.plot_hist(df=df_no_duplicates,
          feat=feat,
          bins=30,
          filename='../Results/01/Hist-'+feat+'.png')

### 10.7 - Recommendation and other Boolean features

In [ ]:
# Histogram of occurrences of each value of each Boolean feature.
for feat in feats_bool:
    plot_bar.plot_bar(df_no_duplicates,feat,'../Results/01/Bar-'+feat+'.png')

### 10.8 - Relationship between recommendation and review score

In [ ]:
df_recommended = df_no_duplicates[df_no_duplicates['recommended']==True].copy()
df_not_recommended = df_no_duplicates[df_no_duplicates['recommended']==False].copy()

In [ ]:
plot_two_hists_comp_sns.plot_two_hists_comp_sns(df_1=df_recommended,
                                                df_2=df_not_recommended,
                                                label_1='Recommended',
                                                label_2='Not recommended',
                                                feat='review_score',
                                                bins=10,
                                                title='Distribution of all customer reviews',
                                                x_label='Review score',
                                                y_label='Entries / bin',
                                                filename='../Results/01/HistRecAndNotRec.png'
                                               )

### 10.9 - Relationship between review score and length of the review text

We calculated the length of the review text in terms of number of characters.

Positive customer reviews tend to be shorter than negative customer reviews (see mean of the distribution of the number of characters in the customer review). The same trend is observed if using the number of words instead of the number of characters.

In [ ]:
plot_two_hists_comp_sns.plot_two_hists_comp_sns(df_1=df_no_duplicates[df_no_duplicates['review_score']<=4],
                        df_2=df_no_duplicates[df_no_duplicates['review_score']>=7],
                        label_1='review score <= 4',
                        label_2='review score >= 7',
                        feat='review_characters',
                        bins=30,
                        title='Distribution of all customer reviews',
                        x_label='Number of characters in the customer review',
                        y_label='Entries / bin',
                        filename='../Results/01/HistReviewCharactersByReviewScore.png')

## 11 - Save the dataset

In [ ]:
df_no_duplicates_types = df_no_duplicates.dtypes.to_frame('dtypes').reset_index()

In [ ]:
df_no_duplicates.to_csv('../Results/PreprocessedDataLight.csv')
df_no_duplicates_types.to_csv('../Results/PreprocessedDataLightTypes.csv')